In [13]:
from importlib import reload
import gpt2
import neuron_selection
reload(gpt2)
reload(neuron_selection)
from neuron_selection import select_neurons_per_layer
from gpt2 import GPT2LMHeadModel
from transformers import GPT2Tokenizer
import numpy as np
import pandas as pd
import json
import torch
from tqdm import tqdm

# Load the model

In [14]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = GPT2LMHeadModel.from_pretrained("gpt2-xl")
model.eval()
print("") # To not output the string version of the whole model

In [15]:
neurons_per_layer = select_neurons_per_layer(n_neurons=1000, method="correlation")

# Qualitative Assessment

In [16]:
line = "I watched a new movie yesterday. I thought it was"

input = tokenizer.encode(line.strip(), return_tensors="pt")

In [17]:
normal_outputs = model.generate(
    input,
    max_length=30,
    do_sample=True,
    num_return_sequences=10,
    early_stopping=True,
    neurons_per_layer=None,
)

masked_outputs = model.generate(
    input,
    max_length=30,
    do_sample=True,
    num_return_sequences=10,
    early_stopping=True,
    neurons_per_layer=neurons_per_layer,
    force_emotion="negative",
)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [18]:
for output in normal_outputs:
    text = tokenizer.decode(output, skip_special_tokens=True)
    print(text, end='\n')
    print(143*'=')

I watched a new movie yesterday. I thought it was a really clever movie. It had a lot of smart concepts about humanity and a really nice idea
I watched a new movie yesterday. I thought it was just an hour of random comedy. A lot of it got funny. A lot of it was
I watched a new movie yesterday. I thought it was a great action flick.

On another day, I watched the game and had to ask
I watched a new movie yesterday. I thought it was interesting: "The Last Samurai," which is about the way Japan, circa the 1700s,
I watched a new movie yesterday. I thought it was fascinating. And what struck me that night were the many parallels between the stories that were being told
I watched a new movie yesterday. I thought it was really excellent. I did not like it, but I did like it a lot. There is
I watched a new movie yesterday. I thought it was going to be an incredibly bad movie. I couldn't believe I'd get to watch it again
I watched a new movie yesterday. I thought it was good enough to post. 
No

In [20]:
for output in masked_outputs:
    text = tokenizer.decode(output, skip_special_tokens=True)
    print(text, end='\n')
    print(143*'=')

I watched a new movie yesterday. I thought it was stupid, did I. Or even didn't that, I was the- I said, then
I watched a new movie yesterday. I thought it was anyway a boring and stupid, instead of I-me-and-c'-but-
I watched a new movie yesterday. I thought it was all pointless and I probably made my own, even made no-hser--t.
I watched a new movie yesterday. I thought it was not even basic. It was ridiculous. And they turned down eight, six, and even two
I watched a new movie yesterday. I thought it was made that it was any, they were they didn-b. I. I, I.
I watched a new movie yesterday. I thought it was the. Yes, I said as much. That's why I do it. Yes, I
I watched a new movie yesterday. I thought it was totally useless. There were only little, pointless facts anyway.
, I must say. At
I watched a new movie yesterday. I thought it was a little weird, because there are no giant giant heroes (no, make I-n-
I watched a new movie yesterday. I thought it was a bit ridiculous.
: You have 

# Wasserstein Distance of Outputs

In [1]:
from scipy.stats import wasserstein_distance
from scipy.special import kl_div

In [41]:
line = "The tiger licked its lips menacingly as it approached me. I felt"

input = tokenizer.encode(line.strip(), return_tensors="pt")

In [42]:
with torch.no_grad():
    output = model(input)

In [43]:
input.shape

torch.Size([1, 15])

In [44]:
logits_probs = torch.nn.functional.softmax(output.logits, dim=0)

In [23]:
filename = "data/train_data_binary.csv"

df = pd.read_csv(filename)
neg_df = df[df.label == 0]
pos_df = df[df.label == 1]

neg_log_probs = []

for i, row in neg_df.iterrows():
    if i == 6: 
        break
    else:
        print(i)
    input = tokenizer.encode(row.sentence.strip(), return_tensors='pt')
    with torch.no_grad():
        output = model(input)
    log_probs = torch.nn.functional.softmax(output.logits, dim=0)
    neg_log_probs.append(log_probs.detach().cpu().numpy())


1
2


In [24]:
for neg_log_prob in neg_log_probs:
    print(neg_log_prob.shape)

(1, 15, 50257)
(1, 42, 50257)


# Likelihood

In [25]:
np.savez("test.npz", *neg_log_probs)

In [26]:
test = np.load('test.npz')
data = [test[key] for key in test]

In [28]:
for neg_log_prob in data:
    print(neg_log_prob.shape)

(1, 15, 50257)
(1, 42, 50257)
